<a href="https://colab.research.google.com/github/Emmcho/ContractElementExtraction/blob/master/ProjectCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pwd

'/content'

In [0]:
import glob
import pandas as pd


In [98]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
ele_c_tr=[]
path = "/content/drive/My Drive/ContractDataset/elements_contracts/train/*.stabilo"
files = glob.glob(path)   
for file in files:     
    f=open(file, 'r') 
    ele_c_tr.append(f.readlines())
    f.close()

In [0]:
ele_c_tr=[[j.split() for j in i] for i in ele_c_tr]

In [0]:
for i in ele_c_tr:
    for j in i:
        j.append("TOKEN_170965[0]")

In [0]:
X=[]
y=[]
for i in ele_c_tr:
    x=[]
    Y=[]
    for j in i:
        for k in j:
            x.append(k[6:k.find('[')])
            Y.append(k[k.find('['):])
    X.append(x)
    y.append(Y)

In [103]:
print (X[0])

['170965', '3840', '2115', '6', '170965', '485', '3840', '2115', '6', '57', '263', '58', '189', '60', '61', '17', '18', '22', '1110', '21', '24', '25', '87', '2984', '19', '30', '19', '31', '87915', '19', '414', '19', '66', '67', '68', '57', '58', '8104', '60', '61', '19', '22', '87916', '4311', '19', '66', '18535', '68', '57', '58', '2131', '60', '61', '252', '170965', '103', '19', '8104', '4755', '87917', '1146', '168', '25', '36130', '29644', '4310', '42954', '7663', '19', '66', '6090', '68', '57', '58', '87918', '60', '61', '57', '26', '58', '286', '60', '61', '19', '3031', '956', '957', '25', '26', '768', '2203', '22', '628', '168', '25', '87918', '102', '170965', '103', '19', '2131', '4755', '87919', '1146', '168', '25', '87918', '19', '3031', '1262', '957', '25', '26', '768', '2203', '22', '628', '168', '25', '87918', '102', '22', '170965', '103', '19', '8104', '4154', '91', '664', '91', '2131', '22', '2131', '4154', '91', '911', '520', '8104', '26', '286', '102', '170965', '240

In [0]:
import pickle

In [0]:
t={'0':0,'TIT':1,'CNP':2,'STD':3,'EFD':4,'TED':5,'PER':6,'VAL':7,'GOV':8,'JUR':9 ,'LEG':10}
y=[[t[j[1:-1]] for j in i] for i in y]

In [0]:
pickle.dump(X,open("Train","wb"))

In [0]:
pickle.dump(y,open("Labels","wb"))

In [0]:
import glob   
ele_c_tr=[]
path = "/content/drive/My Drive/ContractDataset/elements_contracts/test/*.stabilo"
files=glob.glob(path)   
for file in files:     
    f=open(file, 'r') 
    ele_c_tr.append(f.readlines())
    f.close()

In [0]:
ele_c_tr=[[j.split() for j in i] for i in ele_c_tr]

In [0]:
for i in ele_c_tr:
    for j in i:
        if j!= [] and '<' not in j[0]:
            j.append("TOKEN_170965[0]")

In [0]:

for j in range(1,len(ele_c_tr)):
    ele_c_tr[j][0].append("TOKEN_170965[0]")

In [0]:
import numpy as np

In [0]:
zones=[]
X=[]
y=[]
for i in ele_c_tr:
    x=[]
    Y=[]
    z=[]
    for j in i:
        if j != [] and '<' not in j[0]:
            for k in j:
                x.append(k[6:k.find('[')])
                Y.append(k[k.find('['):])
        elif j!=[]:
            z.append(j[0])
            x.append(j[0])
            Y.append(j[0])
            
    X.append(x)
    y.append(Y)
    zones.append(z)

In [0]:
X=[[j for j in i if j.isnumeric() ]for i in X]

In [0]:

y=[[j for j in i if '[' in j]for i in y]

In [0]:
t={'0':0,'TIT':1,'CNP':2,'STD':3,'EFD':4,'TED':5,'PER':6,'VAL':7,'GOV':8,'JUR':9 ,'LEG':10}
y=[[t[j[1:-1]] for j in i] for i in y]

In [0]:
pickle.dump(X,open("Test","wb"))

In [0]:

pickle.dump(y,open("TLabels","wb"))

In [0]:
pickle.dump(zones,open("Tzones","wb"))

In [124]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-z186e4u0
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-z186e4u0
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=7b2cfd093795bb2667ca4b0920c1de2a63bb50c5d4dde6657f96291f3277e675
  Stored in directory: /tmp/pip-ephem-wheel-cache-g7lzycdp/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
import keras

from keras.layers import LSTM,Embedding,Bidirectional,TimeDistributed,Dense
from keras_contrib.layers import CRF
import pickle
from keras.optimizers import Adam
import numpy as np
from keras.layers import Input
from keras.models import Model
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
import pandas as pd

In [0]:
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, tr, tr_y, batch_size=1,n_classes=12, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.labels = tr_y
        self.X=tr
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.X)

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index]

        # Generate data
        X, y = self.__data_generation(indexes)
        return np.array([X]), np.array([y])

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.X))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = self.X[indexes]
        X=[170965 if j==514370 else j for j in X]
        #print(X.shape)
        y = keras.utils.to_categorical(self.labels[indexes], num_classes=self.n_classes)
        #print(y.shape)
        
        return X,y

In [0]:
class TDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, tr, batch_size=1,shuffle=False):
        'Initialization'
        self.batch_size = batch_size
        self.X=tr
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.X)

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index]

        # Generate data
        X = self.__data_generation(indexes)
        return np.array([X])

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.X))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = self.X[indexes]
        X=[170965 if j==514370 else j for j in X]
        #print(X.shape)
            
        return X

In [0]:
t={'0':0,'TIT':1,'CNP':2,'STD':3,'EFD':4,'TED':5,'PER':6,'VAL':7,'GOV':8,'JUR':9 ,'LEG':10,'HEAD':11}

In [0]:
max_len=69593
vocab_size=170965
n_tags=len(t)

In [0]:
file=open("Train","rb")
tr=pickle.load(file)

In [0]:
file=open("Labels","rb")
tr_y=pickle.load(file)

In [0]:
#ourdict=[]
#path = "/content/drive/My Drive/ContractDataset/embd_wt.dms"
#f=open(path, 'r') 
#ourdict.append(f.readlines())
#f.close()  
#wt=pickle.load(open("embd_wt","rb"))

wt=[]
path = "/content/drive/My Drive/ContractDataset/embd_wt.dms"
files=glob.glob(path)   
for file in files:     
    f=open(file, 'rb') 
    wt.append(f.readlines())
    f.close()

In [0]:
import pandas as pd

wt = pd.read_pickle(r'/content/drive/My Drive/ContractDataset/embd_wt.dms')

In [0]:
wt=np.append([np.array([0 for i in range(225)])],wt,axis=0)

In [0]:
input = Input(shape=(max_len,))
#input = input.get_shape(69593,31224).T
#input = input.get_shape(input,[69593,31224]).T
model = Embedding(input_dim=vocab_size+1, output_dim=225,weights=[wt],
                  input_length=max_len, mask_zero=False)(input)
model = Bidirectional(LSTM(units=50, return_sequences=True,recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)   #a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)




In [184]:
input

<tf.Tensor 'input_19:0' shape=(None, 69593) dtype=float32>

In [0]:
model = Model(input, out)

In [186]:
model.compile(optimizer="adam", loss=crf.loss_function, metrics=[crf.accuracy])

/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [187]:
model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 69593)             0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 69593, 225)        38467350  
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 69593, 100)        110400    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 69593, 50)         5050      
_________________________________________________________________
crf_10 (CRF)                 (None, 69593, 12)         780       
Total params: 38,583,580
Trainable params: 38,583,580
Non-trainable params: 0
_________________________________________________________________


In [0]:
gen=DataGenerator(tr,tr_y)

In [189]:
print (tr_y[5])

[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [0]:

 tr_y=np.array(tr_y)
 tr_y.shape

In [190]:
history = model.fit_generator(generator=gen, epochs=1,verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1


ValueError: ignored